In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from scipy.stats import zscore

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

df = pd.read_csv('resources/train.csv')
df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 56 columns):
Id                                    15120 non-null int64
Elevation                             15120 non-null int64
Aspect                                15120 non-null int64
Slope                                 15120 non-null int64
Horizontal_Distance_To_Hydrology      15120 non-null int64
Vertical_Distance_To_Hydrology        15120 non-null int64
Horizontal_Distance_To_Roadways       15120 non-null int64
Hillshade_9am                         15120 non-null int64
Hillshade_Noon                        15120 non-null int64
Hillshade_3pm                         15120 non-null int64
Horizontal_Distance_To_Fire_Points    15120 non-null int64
Wilderness_Area1                      15120 non-null int64
Wilderness_Area2                      15120 non-null int64
Wilderness_Area3                      15120 non-null int64
Wilderness_Area4                      15120 non-null int64
Soil_T

In [58]:
numerical_features = df[['Elevation', 'Aspect', 'Slope', "Horizontal_Distance_To_Hydrology",
                         'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                         'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']]
numerical_features = numerical_features.apply(zscore)
numerical_features.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,4.334805
1,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,4.285710
2,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,4.191156
3,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,4.272981
4,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,4.237524


In [59]:
categorical_features = df.select(lambda col: col.startswith('Wilderness_Area') or col.startswith('Soil_Type'), axis=1).copy()
categorical_features.head()

,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
train_features = pd.concat([numerical_features, categorical_features], axis = 1).values
train_features

array([[-0.36709512, -0.95997982, -1.59713163, ...,  0.        ,
         0.        ,  0.        ],
       [-0.38146072, -0.9145592 , -1.71542377, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.1309124 , -0.16057691, -0.8873788 , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.61609888, -0.20599753,  1.00529541, ...,  0.        ,
         0.        ,  0.        ],
       [-0.62807021,  0.09377856,  1.36017182, ...,  0.        ,
         0.        ,  0.        ],
       [-0.65680142,  0.36630228,  2.06992465, ...,  0.        ,
         0.        ,  0.        ]])

In [37]:
train_target = pd.get_dummies(df['Cover_Type']).values
train_target

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

In [67]:
model = Sequential([
    Dense(256, input_shape=train_features.shape[1:], activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_target.shape[1], activation='softmax')
])

model.compile(optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])


model.fit(train_features, train_target, 
             batch_size=120, epochs=1000, verbose=1, 
             validation_split=0.15, 
             callbacks=[
                ModelCheckpoint("forest.h5", save_best_only=True),
                EarlyStopping(patience=40)
             ])

score = model.evaluate(train_features, train_target)
print('\n\n')
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train on 12852 samples, validate on 2268 samples
Epoch 1/1000
12852/12852 [==============================] - 0s - loss: 1.9490 - acc: 0.1462 - val_loss: 1.9475 - val_acc: 0.1570
Epoch 2/1000
12852/12852 [==============================] - 0s - loss: 1.9479 - acc: 0.1478 - val_loss: 1.9474 - val_acc: 0.1526
Epoch 3/1000
12852/12852 [==============================] - 0s - loss: 1.9468 - acc: 0.1478 - val_loss: 1.9472 - val_acc: 0.1464
Epoch 4/1000
12852/12852 [==============================] - 0s - loss: 1.9458 - acc: 0.1473 - val_loss: 1.9470 - val_acc: 0.1407
Epoch 5/1000
12852/12852 [==============================] - 0s - loss: 1.9446 - acc: 0.1544 - val_loss: 1.9468 - val_acc: 0.1389
Epoch 6/1000
12852/12852 [==============================] - 0s - loss: 1.9428 - acc: 0.1600 - val_loss: 1.9465 - val_acc: 0.1354
Epoch 7/1000
12852/12852 [==============================] - 0s - loss: 1.9425 - acc: 0.1569 - val_loss: 1.9463 - val_acc: 0.1376
Epoch 8/1000
12852/12852 [======================

12852/12852 [==============================] - 0s - loss: 1.5649 - acc: 0.3247 - val_loss: 1.5190 - val_acc: 0.3761
Epoch 64/1000
12852/12852 [==============================] - 0s - loss: 1.5464 - acc: 0.3363 - val_loss: 1.5043 - val_acc: 0.3761
Epoch 65/1000
12852/12852 [==============================] - 0s - loss: 1.5392 - acc: 0.3350 - val_loss: 1.4897 - val_acc: 0.3792
Epoch 66/1000
12852/12852 [==============================] - 0s - loss: 1.5300 - acc: 0.3326 - val_loss: 1.4758 - val_acc: 0.3871
Epoch 67/1000
12852/12852 [==============================] - 0s - loss: 1.5165 - acc: 0.3385 - val_loss: 1.4625 - val_acc: 0.3898
Epoch 68/1000
12852/12852 [==============================] - 0s - loss: 1.5042 - acc: 0.3354 - val_loss: 1.4492 - val_acc: 0.3959
Epoch 69/1000
12852/12852 [==============================] - 0s - loss: 1.4896 - acc: 0.3424 - val_loss: 1.4360 - val_acc: 0.4017
Epoch 70/1000
12852/12852 [==============================] - 0s - loss: 1.4877 - acc: 0.3445 - val_loss:

12852/12852 [==============================] - 0s - loss: 1.1465 - acc: 0.4883 - val_loss: 1.0329 - val_acc: 0.5450
Epoch 126/1000
12852/12852 [==============================] - 0s - loss: 1.1393 - acc: 0.4943 - val_loss: 1.0272 - val_acc: 0.5476
Epoch 127/1000
12852/12852 [==============================] - 0s - loss: 1.1303 - acc: 0.4991 - val_loss: 1.0225 - val_acc: 0.5494
Epoch 128/1000
12852/12852 [==============================] - 0s - loss: 1.1325 - acc: 0.4983 - val_loss: 1.0193 - val_acc: 0.5454
Epoch 129/1000
12852/12852 [==============================] - 0s - loss: 1.1312 - acc: 0.4963 - val_loss: 1.0145 - val_acc: 0.5485
Epoch 130/1000
12852/12852 [==============================] - 0s - loss: 1.1141 - acc: 0.5021 - val_loss: 1.0125 - val_acc: 0.5507
Epoch 131/1000
12852/12852 [==============================] - 0s - loss: 1.1218 - acc: 0.5065 - val_loss: 1.0076 - val_acc: 0.5525
Epoch 132/1000
12852/12852 [==============================] - 0s - loss: 1.1067 - acc: 0.5102 - va

12852/12852 [==============================] - 0s - loss: 0.9458 - acc: 0.5809 - val_loss: 0.8664 - val_acc: 0.6323
Epoch 188/1000
12852/12852 [==============================] - 0s - loss: 0.9460 - acc: 0.5868 - val_loss: 0.8664 - val_acc: 0.6314
Epoch 189/1000
12852/12852 [==============================] - 0s - loss: 0.9353 - acc: 0.5906 - val_loss: 0.8615 - val_acc: 0.6371
Epoch 190/1000
12852/12852 [==============================] - 0s - loss: 0.9479 - acc: 0.5818 - val_loss: 0.8613 - val_acc: 0.6376
Epoch 191/1000
12852/12852 [==============================] - 0s - loss: 0.9468 - acc: 0.5823 - val_loss: 0.8587 - val_acc: 0.6389
Epoch 192/1000
12852/12852 [==============================] - 0s - loss: 0.9329 - acc: 0.5891 - val_loss: 0.8591 - val_acc: 0.6380
Epoch 193/1000
12852/12852 [==============================] - 0s - loss: 0.9339 - acc: 0.5904 - val_loss: 0.8569 - val_acc: 0.6380
Epoch 194/1000
12852/12852 [==============================] - 0s - loss: 0.9339 - acc: 0.5891 - va

12852/12852 [==============================] - 0s - loss: 0.8475 - acc: 0.6358 - val_loss: 0.7948 - val_acc: 0.6742
Epoch 250/1000
12852/12852 [==============================] - 0s - loss: 0.8421 - acc: 0.6332 - val_loss: 0.7977 - val_acc: 0.6728
Epoch 251/1000
12852/12852 [==============================] - 0s - loss: 0.8458 - acc: 0.6354 - val_loss: 0.7934 - val_acc: 0.6737
Epoch 252/1000
12852/12852 [==============================] - 0s - loss: 0.8427 - acc: 0.6384 - val_loss: 0.7919 - val_acc: 0.6750
Epoch 253/1000
12852/12852 [==============================] - 0s - loss: 0.8406 - acc: 0.6313 - val_loss: 0.7925 - val_acc: 0.6742
Epoch 254/1000
12852/12852 [==============================] - 0s - loss: 0.8425 - acc: 0.6352 - val_loss: 0.7906 - val_acc: 0.6750
Epoch 255/1000
12852/12852 [==============================] - 0s - loss: 0.8301 - acc: 0.6398 - val_loss: 0.7874 - val_acc: 0.6737
Epoch 256/1000
12852/12852 [==============================] - 0s - loss: 0.8327 - acc: 0.6337 - va

...

12852/12852 [==============================] - 0s - loss: 0.5336 - acc: 0.7878 - val_loss: 0.5831 - val_acc: 0.7703
Epoch 746/1000
12852/12852 [==============================] - 0s - loss: 0.5351 - acc: 0.7884 - val_loss: 0.5897 - val_acc: 0.7690
Epoch 747/1000
12852/12852 [==============================] - 0s - loss: 0.5340 - acc: 0.7856 - val_loss: 0.5929 - val_acc: 0.7668
Epoch 748/1000
12852/12852 [==============================] - 0s - loss: 0.5383 - acc: 0.7842 - val_loss: 0.5868 - val_acc: 0.7694
Epoch 749/1000
12852/12852 [==============================] - 0s - loss: 0.5323 - acc: 0.7857 - val_loss: 0.5840 - val_acc: 0.7694
Epoch 750/1000
12852/12852 [==============================] - 0s - loss: 0.5326 - acc: 0.7877 - val_loss: 0.5805 - val_acc: 0.7694
Epoch 751/1000
12852/12852 [==============================] - 0s - loss: 0.5339 - acc: 0.7845 - val_loss: 0.5893 - val_acc: 0.7650
Epoch 752/1000
12852/12852 [==============================] - 0s - loss: 0.5322 - acc: 0.7907 - va

12852/12852 [==============================] - 0s - loss: 0.5060 - acc: 0.7993 - val_loss: 0.5671 - val_acc: 0.7751
Epoch 808/1000
12852/12852 [==============================] - 0s - loss: 0.5101 - acc: 0.7978 - val_loss: 0.5775 - val_acc: 0.7734
Epoch 809/1000
12852/12852 [==============================] - 0s - loss: 0.5070 - acc: 0.7972 - val_loss: 0.5644 - val_acc: 0.7760
Epoch 810/1000
12852/12852 [==============================] - 0s - loss: 0.5046 - acc: 0.7986 - val_loss: 0.5856 - val_acc: 0.7694
Epoch 811/1000
12852/12852 [==============================] - 0s - loss: 0.4995 - acc: 0.8027 - val_loss: 0.5796 - val_acc: 0.7738
Epoch 812/1000
12852/12852 [==============================] - 0s - loss: 0.5059 - acc: 0.7965 - val_loss: 0.5741 - val_acc: 0.7751
Epoch 813/1000
12852/12852 [==============================] - 0s - loss: 0.5023 - acc: 0.8035 - val_loss: 0.5707 - val_acc: 0.7773
Epoch 814/1000
12852/12852 [==============================] - 0s - loss: 0.5036 - acc: 0.7995 - va

12852/12852 [==============================] - 0s - loss: 0.4731 - acc: 0.8126 - val_loss: 0.5573 - val_acc: 0.7791
Epoch 870/1000
12852/12852 [==============================] - 0s - loss: 0.4706 - acc: 0.8162 - val_loss: 0.5631 - val_acc: 0.7813
Epoch 871/1000
12852/12852 [==============================] - 0s - loss: 0.4739 - acc: 0.8163 - val_loss: 0.5706 - val_acc: 0.7804
Epoch 872/1000
12852/12852 [==============================] - 0s - loss: 0.4722 - acc: 0.8150 - val_loss: 0.5539 - val_acc: 0.7831
Epoch 873/1000
12852/12852 [==============================] - 0s - loss: 0.4775 - acc: 0.8111 - val_loss: 0.5602 - val_acc: 0.7822
Epoch 874/1000
12852/12852 [==============================] - 0s - loss: 0.4735 - acc: 0.8148 - val_loss: 0.5676 - val_acc: 0.7844
Epoch 875/1000
12852/12852 [==============================] - 0s - loss: 0.4747 - acc: 0.8096 - val_loss: 0.5691 - val_acc: 0.7835
Epoch 876/1000
12852/12852 [==============================] - 0s - loss: 0.4738 - acc: 0.8147 - va

12852/12852 [==============================] - 0s - loss: 0.4477 - acc: 0.8296 - val_loss: 0.5570 - val_acc: 0.7892
Epoch 932/1000
12852/12852 [==============================] - 0s - loss: 0.4488 - acc: 0.8288 - val_loss: 0.5562 - val_acc: 0.7875
Epoch 933/1000
12852/12852 [==============================] - 0s - loss: 0.4423 - acc: 0.8253 - val_loss: 0.5456 - val_acc: 0.7901
Epoch 934/1000
12852/12852 [==============================] - 0s - loss: 0.4457 - acc: 0.8318 - val_loss: 0.5486 - val_acc: 0.7906
Epoch 935/1000
12852/12852 [==============================] - 0s - loss: 0.4478 - acc: 0.8289 - val_loss: 0.5485 - val_acc: 0.7919
Epoch 936/1000
12852/12852 [==============================] - 0s - loss: 0.4424 - acc: 0.8288 - val_loss: 0.5428 - val_acc: 0.7844
Epoch 937/1000
12852/12852 [==============================] - 0s - loss: 0.4404 - acc: 0.8301 - val_loss: 0.5440 - val_acc: 0.7879
Epoch 938/1000
12852/12852 [==============================] - 0s - loss: 0.4343 - acc: 0.8340 - va

12852/12852 [==============================] - 0s - loss: 0.4161 - acc: 0.8414 - val_loss: 0.5342 - val_acc: 0.7954
Epoch 994/1000
12852/12852 [==============================] - 0s - loss: 0.4137 - acc: 0.8420 - val_loss: 0.5653 - val_acc: 0.7892
Epoch 995/1000
12852/12852 [==============================] - 0s - loss: 0.4186 - acc: 0.8407 - val_loss: 0.5360 - val_acc: 0.7967
Epoch 996/1000
12852/12852 [==============================] - 0s - loss: 0.4132 - acc: 0.8445 - val_loss: 0.5470 - val_acc: 0.7963
Epoch 997/1000
12852/12852 [==============================] - 0s - loss: 0.4094 - acc: 0.8441 - val_loss: 0.5700 - val_acc: 0.7870
Epoch 998/1000
12852/12852 [==============================] - 0s - loss: 0.4142 - acc: 0.8459 - val_loss: 0.5390 - val_acc: 0.7985
Epoch 999/1000
12852/12852 [==============================] - 0s - loss: 0.4107 - acc: 0.8459 - val_loss: 0.5331 - val_acc: 0.7945
Epoch 1000/1000
14048/15120 [==========================>...] - ETA: 0s


Train loss: 0.36302349253

In [68]:
np.argmax(train_target, axis=1) + 1

array([5, 5, 2, ..., 3, 3, 3])